In [3]:
%%configure -f
{ "conf": {
            "spark.jars":"hdfs:///apps/hudi/lib/hudi-spark-bundle.jar,hdfs:///apps/hudi/lib/spark-avro.jar",
            "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
            "spark.sql.hive.convertMetastoreParquet":"false"
          }}

UsageError: Cell magic `%%configure` not found.


In [2]:
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor
import org.apache.hudi.QuickstartUtils._
import scala.collection.JavaConversions._
import org.apache.spark.sql.SaveMode._
import org.apache.hudi.DataSourceReadOptions._
import org.apache.hudi.DataSourceWriteOptions._
import org.apache.hudi.config.HoodieWriteConfig._

ModuleNotFoundError: No module named 'org'

In [5]:
val inputDataPath = "s3://octank-datalake-seba/hstaging/home_loan/credit_card_balance/"
val hudiTableName = "credit_card_balance"
val hudiTablePath = "s3://octank-datalake-seba/hstandardization/home_loan/" + hudiTableName

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

inputDataPath: String = s3://octank-datalake-seba/test/
hudiTableName: String = credit_card_balance
hudiTablePath: String = s3://octank-datalake-seba/test2/credit_card_balance


In [6]:
val inputDF = spark.read.option("header","true").parquet(inputDataPath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

inputDF: org.apache.spark.sql.DataFrame = [SK_ID_PREV: string, SK_ID_CURR: string ... 21 more fields]


In [7]:
inputDF.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res1: Long = 3840312


In [8]:
inputDF.select("SK_ID_PREV").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res2: Long = 104307


In [9]:
inputDF.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+
|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT|CNT_DRAWINGS_CURRENT|CNT_DRAWINGS_OTHER_CURRENT|CNT_DRAWINGS_POS_CURRENT|CNT_INSTALMENT_MATURE_CUM|NAME_CONTRACT_STATUS|SK_DPD|SK_DPD_DEF|
+----------+----------+--------------+-----------+------------------

In [10]:
val hudiOptions = Map[String,String](
    HoodieWriteConfig.TABLE_NAME -> hudiTableName,
    DataSourceWriteOptions.TABLE_TYPE_OPT_KEY -> "COPY_ON_WRITE",
    DataSourceWriteOptions.RECORDKEY_FIELD_OPT_KEY -> "SK_ID_PREV", 
    DataSourceWriteOptions.PRECOMBINE_FIELD_OPT_KEY -> "SK_ID_PREV", 
    DataSourceWriteOptions.HIVE_SYNC_ENABLED_OPT_KEY -> "true", 
    DataSourceWriteOptions.HIVE_TABLE_OPT_KEY -> hudiTableName)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

hudiOptions: scala.collection.immutable.Map[String,String] = Map(hoodie.datasource.write.precombine.field -> SK_ID_PREV, hoodie.datasource.hive_sync.table -> credit_card_balance, hoodie.datasource.hive_sync.enable -> true, hoodie.datasource.write.recordkey.field -> SK_ID_PREV, hoodie.table.name -> credit_card_balance)


In [11]:
inputDF.write.format("org.apache.hudi").
    option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.INSERT_OPERATION_OPT_VAL).
    options(hudiOptions).
    mode(SaveMode.Overwrite).
    save(hudiTablePath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#val snapshotQueryDF = spark.read.format("org.apache.hudi").load("s3://octank-datalake-seba/test2/" + "/*/*")
val snapshotQueryDF = spark.read.format("org.apache.hudi").load("s3://octank-datalake-seba/test2/*/*")
snapshotQueryDF.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

snapshotQueryDF: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 26 more fields]
res6: Long = 104307


In [13]:
snapshotQueryDF.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res7: Long = 104307


In [14]:
#1891521,1389973,1740877,2582071,2562384

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
snapshotQueryDF.createOrReplaceTempView("credit_card_balance")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
val df_del = spark.sql("select * from credit_card_balance where SK_ID_PREV = '1891521' or SK_ID_PREV = '1389973'")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df_del: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 26 more fields]


In [20]:
df_del.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------------+------------------+----------------------+--------------------+----------+----------+--------------+-----------+-----------------------+------------------------+--------------------+--------------------------+------------------------+-----------------------+-------------------+-------------------------+------------------------+-------------+--------------------+------------------------+--------------------+--------------------------+------------------------+-------------------------+--------------------+------+----------+
|_hoodie_commit_time|_hoodie_commit_seqno|_hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|SK_ID_PREV|SK_ID_CURR|MONTHS_BALANCE|AMT_BALANCE|AMT_CREDIT_LIMIT_ACTUAL|AMT_DRAWINGS_ATM_CURRENT|AMT_DRAWINGS_CURRENT|AMT_DRAWINGS_OTHER_CURRENT|AMT_DRAWINGS_POS_CURRENT|AMT_INST_MIN_REGULARITY|AMT_PAYMENT_CURRENT|AMT_PAYMENT_TOTAL_CURRENT|AMT_RECEIVABLE_PRINCIPAL|AMT_RECIVABLE|AMT_TOTAL_RECEIVABLE|CNT_DRAWINGS_ATM_CURRENT

In [21]:
df_del.write.
format("org.apache.hudi").
options(hudiOptions).
option(DataSourceWriteOptions.OPERATION_OPT_KEY,DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL).
option(DataSourceWriteOptions.PAYLOAD_CLASS_OPT_KEY,"org.apache.hudi.common.model.EmptyHoodieRecordPayload").
mode(SaveMode.Append).
save(hudiTablePath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
val snapshotQueryDF = spark.read.format("org.apache.hudi").load("s3://octank-datalake-seba/test2/*/*")
snapshotQueryDF.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

snapshotQueryDF: org.apache.spark.sql.DataFrame = [_hoodie_commit_time: string, _hoodie_commit_seqno: string ... 26 more fields]
res15: Long = 104305
